<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/CaptumLongformerSequenceClassificationPapersMultiembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [3]:
pip install transformers --quiet

In [4]:
pip install captum --quiet

In [5]:
pip install datasets --quiet

In [6]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = '/content/drive/MyDrive/cogs402longformer/models/longformer-finetuned_papers/checkpoint-2356'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

In [10]:
model2 = BertForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='allenai/longformer-base-4096', vocab_size=50265, model_max_len=4096, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})


In [23]:
def predict(inputs, position_ids=None):
    output = model(inputs,
                   position_ids=position_ids)
    return output.logits

In [13]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [14]:
max_length = 2048
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [15]:
from datasets import load_dataset
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-cc784554b797f843
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-cc784554b797f843/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
testval = 923
text = cogs402_ds['text'][testval]
label = cogs402_ds['labels'][testval]
print(label)

0


In [17]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   )
    return torch.softmax(preds, dim = 1)[:, 0] # for negative attribution, 
    #return torch.softmax(preds, dim = 1)[:, 1] #<- for positive attribution

In [18]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [19]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [20]:
print(position_ids)
print(ref_position_ids)

tensor([[   0,    1,    2,  ..., 2047, 2048, 2049]], device='cuda:0')
tensor([[0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')


In [21]:
lig2 = LayerIntegratedGradients(custom_forward, \
                                [model.longformer.embeddings.word_embeddings, \
                                 model.longformer.embeddings.position_embeddings])

/usr/local/lib/python3.7/dist-packages/captum/attr/_core/layer/layer_integrated_gradients.py:103: UserWarning: Multiple layers provided. Please ensure that each layer is**not** solely solely dependent on the outputs ofanother layer. Please refer to the documentation for moredetail.
  "Multiple layers provided. Please ensure that each layer is"


In [24]:
attributions2 = lig2.attribute(inputs=(input_ids, position_ids),
                               baselines=(ref_input_ids, ref_position_ids),
                               n_steps=50,
                               internal_batch_size = 2)

In [26]:
attributions_word = summarize_attributions(attributions2[0])
attributions_position = summarize_attributions(attributions2[1])

In [27]:
def get_topk_attributed_tokens(attrs, k=15):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    print(values)
    return top_tokens, values, indices

In [28]:
def get_botk_attributed_tokens(attrs, k=15):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [29]:
import pandas as pd
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_word)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_word)


top_pos_start, top_pos_val_start, pos_ind_start = get_topk_attributed_tokens(attributions_position)
bot_pos_start, bot_pos_val_start, pos_ind_start2 = get_botk_attributed_tokens(attributions_position)

df_high = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words_start, top_word_ind_start, top_words_val_start)],
                   'Position(Index), Attribution': ["{} ({}), {}".format(position, pos, round(val.item(),2)) for position, pos, val in zip(top_pos_start, pos_ind_start, top_pos_val_start)]})

df_low = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(bot_words_start, bot_word_ind_start, bot_words_val_start)],
                   'Position(Index), Attribution': ["{} ({}), {}".format(position, pos, round(val.item(),2)) for position, pos, val in zip(bot_pos_start, pos_ind_start2, bot_pos_val_start)]})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

tensor([0.2440, 0.2405, 0.2173, 0.2056, 0.1914, 0.1913, 0.1865, 0.1709, 0.1700,
        0.1446, 0.1437, 0.1430, 0.1328, 0.1252, 0.1222], device='cuda:0',
       dtype=torch.float64)
tensor([0.3204, 0.1947, 0.1211, 0.0836, 0.0520, 0.0449, 0.0447, 0.0395, 0.0370,
        0.0368, 0.0313, 0.0308, 0.0279, 0.0277, 0.0276], device='cuda:0',
       dtype=torch.float64)


In [30]:
df_high

,"Word(Index), Attribution","Position(Index), Attribution"
0,"Ġprogramming (334), 0.24",", (514), 0.32"
1,"Ġprogramming (314), 0.24",", (529), 0.19"
2,", (529), 0.22","ĠAlso (663), 0.12"
3,"Ġprogramming (171), 0.21","X (528), 0.08"
4,"Ġprogramming (84), 0.19","- (1362), 0.05"
5,"Ġsyntax (195), 0.19","Ġsize (117), 0.04"
6,", (514), 0.19",", (664), 0.04"
7,"Ġsyntax (263), 0.17","Ġbe (818), 0.04"
8,"ĠThe (116), 0.17",". (1373), 0.04"
9,"Ġprogramming (445), 0.14","ĠIts (346), 0.04"


In [31]:
df_low

,"Word(Index), Attribution","Position(Index), Attribution"
0,"ĠTo (151), -0.17","ĠC (1538), -0.72"
1,"ĠThe (468), -0.09","opt (2), -0.37"
2,"utions (1891), -0.09","ĠTo (151), -0.18"
3,"ative (873), -0.08","Ġlanguage (356), -0.05"
4,"ative (822), -0.07",", (154), -0.03"
5,"Ġis (142), -0.06",", (397), -0.03"
6,"Ġtime (642), -0.06","Ġend (153), -0.03"
7,"Ġsolving (148), -0.06","Ġsyntax (195), -0.02"
8,"Ġexpert (231), -0.05","Ġvariable (108), -0.02"
9,"Ġtime (654), -0.05","Ġfocus (1898), -0.02"


In [32]:
d = {"tokens":all_tokens, "attribution":attributions_word[:2050].cpu()}
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [33]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False)[:15]
highest_attrib_tokens

,attribution
tokens,
Ġprogramming,1.262697
",",0.847716
Ġsyntax,0.557280
Ġto,0.444532
Ġ,0.366207
Ġthe,0.361825
Ġlanguage,0.352078
Ġand,0.323564
Ġ.,0.302341


In [34]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution'])[:15]
lowest_attrib_tokens

,attribution
tokens,
.,-0.516403
-,-0.285388
Ġa,-0.267804
Ġof,-0.246206
ative,-0.238763
ĠASP,-0.228577
Ġ(,-0.198362
ĠTo,-0.164541
"),",-0.146611
